# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias

import sys
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import seaborn as sns
import warnings
import mlflow
import mlflow.xgboost
import mlflow.sklearn
import dagshub

from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from src.config.paths import DADOS_CREDICARD_TRATADO
from src.config.paths import MODELS_DIR

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Conectando o MLflow ao DagsHub
dagshub.init(repo_owner='flaviohenriquehb777', repo_name='Projeto_4_Modelo_de_Identificacao_de_Fraude', mlflow=True)

Accessing as flaviohenriquehb777

Initialized MLflow to track repo "flaviohenriquehb777/Projeto_4_Modelo_de_Identificacao_de_Fraude"

Repository flaviohenriquehb777/Projeto_4_Modelo_de_Identificacao_de_Fraude initialized!

### Funções de Preparação e Treinamento

In [4]:
# Função para carregar e pré-processar os dados
def load_and_preprocess_data(data_path):
    """Carrega e aplica o pré-processamento nos dados."""
    transacoes = pd.read_parquet(data_path)
    transacoes = transacoes.sample(frac=1, random_state=42)
    
    # Criando e aplicando os escaladores
    power_transformer = PowerTransformer(method='yeo-johnson')
    minmax_scaler = MinMaxScaler()
    transacoes['Amount'] = power_transformer.fit_transform(transacoes[['Amount']])
    transacoes['Time'] = minmax_scaler.fit_transform(transacoes[['Time']])
    
    X = transacoes.drop(columns='Class')
    y = transacoes['Class']
    
    return X, y

# Função para treinar o modelo e registrar no MLflow
def train_and_log_model(model_name, model_class, params, X, y):
    """Treina um modelo, calcula as métricas e registra tudo no MLflow."""
    
    with mlflow.start_run(run_name=f"Treinamento do Modelo - {model_name}"):
        mlflow.log_params(params)
        
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        precision_scores = []
        recall_scores = []

        # Loop de validação cruzada
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            model = model_class(**params)
            model.fit(X_train, y_train)
            
            y_pred = model.predict(X_test)
            
            accuracy_scores.append(accuracy_score(y_test, y_pred))
            precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
            recall_scores.append(recall_score(y_test, y_pred, zero_division=0))

        # Cálculo das métricas médias
        mean_accuracy = np.mean(accuracy_scores)
        mean_precision = np.mean(precision_scores)
        mean_recall = np.mean(recall_scores)
        
        # Log das métricas no MLflow
        mlflow.log_metric("mean_accuracy", mean_accuracy)
        mlflow.log_metric("mean_precision", mean_precision)
        mlflow.log_metric("mean_recall", mean_recall)
        
        print(f"Resultados Médios para {model_name}:")
        print(f"Acurácia: {mean_accuracy:.4f}")
        print(f"Precisão: {mean_precision:.4f}")
        print(f"Recall: {mean_recall:.4f}")

        # Registra o modelo usando o sabor correto da biblioteca
        if isinstance(model, xgb.XGBClassifier):
            mlflow.xgboost.log_model(
                xgb_model=model,
                artifact_path="model",
                input_example=X_test.iloc[[0]]
            )
        elif isinstance(model, LogisticRegression):
            mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path="model",
                input_example=X_test.iloc[[0]]
            )
        else:
            print("Aviso: Modelo de tipo desconhecido, não registrado no MLflow.")

### Execução dos Experimentos

In [5]:
# Carrega e pré-processa os dados uma única vez
X, y = load_and_preprocess_data(DADOS_CREDICARD_TRATADO)

In [6]:
# 1. Experimento com o Modelo XGBoost (Configuração atual)
params_xgboost = {
    "max_depth": 9,
    "n_estimators": 200,
    "learning_rate": 0.3,
    "random_state": 0,
    "scale_pos_weight": (y.value_counts()[0] / y.value_counts()[1]),
}
train_and_log_model("XGBoost", xgb.XGBClassifier, params_xgboost, X, y)

Resultados Médios para XGBoost:
Acurácia: 0.9996
Precisão: 0.9227
Recall: 0.8102


In [7]:
# 2. Experimento com o Modelo XGBoost (Hiperparâmetros diferentes)
params_xgboost_2 = {
     "max_depth": 7,
     "n_estimators": 300,
     "learning_rate": 0.1,
     "random_state": 0,
     "scale_pos_weight": (y.value_counts()[0] / y.value_counts()[1]),
}
train_and_log_model("XGBoost Otimizado", xgb.XGBClassifier, params_xgboost_2, X, y)

Resultados Médios para XGBoost Otimizado:
Acurácia: 0.9996
Precisão: 0.9228
Recall: 0.8141


In [8]:
# 3. Experimento com outro modelo (Regressão Logística)
params_logreg = {
    "C": 0.1,
    "solver": 'liblinear',
    "random_state": 42
}
train_and_log_model("LogisticRegression", LogisticRegression, params_logreg, X, y)

Resultados Médios para LogisticRegression:
Acurácia: 0.9992
Precisão: 0.8775
Recall: 0.6067
